# Homework 1

### Research Methods, Spring 2025

**Author**: Pablo Estrada

The answers to the homework questions using Python are described below. Note that I follow closely the analysis from professor McCarthy and his `R` script. My analysis file is also available in the analysis folder.

In [1]:
# Importing the libraries
import pandas as pd

# To run in the terminal:
# python submission1/py-data-code/1_Plan_Data.py
# python submission1/py-data-code/2_Plan_Data.py

# Read output datasets
full_ma_data = pd.read_csv('../data/output/full_ma_data.csv')
contract_service_area = pd.read_csv('../data/output/contract_service_area.csv')

In [20]:
full_ma_data.sample(3)

,contractid,planid,org_type,plan_type,partd,snp,eghp,org_name,org_marketing_name,plan_name,parent_org,contract_date,ssa,fips,state,county,avg_enrollment,year
179071,H1509,804.0,Local CCP,Local PPO,Yes,No,Yes,UNITEDHEALTHCARE INSURANCE COMPANY,UnitedHealthcare,UnitedHealthcare Group Medicare Advantage (PPO),"UnitedHealth Group, Inc.",07/01/2005 0:00:00,26800.0,29161.0,MO,Phelps,NaN,2015
2174752,S7950,804.0,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,Yes,No,Yes,EXPRESS SCRIPTS INSURANCE CO.,Express Scripts Medicare,Express Scripts Medicare (PDP),Express Scripts Holding Company,01/01/2007 0:00:00,42020.0,45005.0,SC,Allendale,NaN,2015
1424574,S3474,826.0,Medicare Prescription Drug Plan,Medicare Prescription Drug Plan,Yes,No,Yes,HEARTLAND FIDELITY INSURANCE COMPANY,Precise RxCare,Precise RxCare (PDP),Heartland Fidelity Insurance Company,01/01/2013 0:00:00,15750.0,18151.0,IN,Steuben,NaN,2015


In [21]:
contract_service_area.sample(3)

,contractid,org_name,org_type,plan_type,partial,eghp,ssa,fips,county,state,notes,year
310948,H7149,"COVENTRY HEALTH CARE OF NEBRASKA, INC.",Local CCP,HMO/HMOPOS,False,False,5340.0,6047.0,Merced,CA,NaN,2015
227562,H4906,C AND O EMPLOYEES' HOSPITAL ASSOCIATION,HCPP - 1833 Cost,HCPP - 1833 Cost,False,False,36460.0,39089.0,Licking,OH,NaN,2015
69087,H1608,COVENTRY HEALTH AND LIFE INSURANCE COMPANY,Local CCP,Local PPO,False,False,52040.0,55009.0,Brown,WI,NaN,2015


**1. How many observations exist in your current dataset?**

First we need to create the enrollment data from Medicare Advantage. Then we just count the total number of plans.

In [29]:
# Total observations and unique combinations of contract/plan/county/year
nobs = full_ma_data.shape[0]
nobs_unique = full_ma_data.drop_duplicates(subset=['contractid', 'planid', 'county', 'year']).shape[0]
print(f"There are {nobs} total observations, which means there are {nobs_unique} unique combinations of contract/plan/county/year.")

There are 2251612 total observations, which means there are 1447946 unique combinations of contract/plan/county/year.


**2. How many different *plan_types* exist in the data?**

To do this, we need to group by plan type and count the number of unique plan types. I did this by creating a table of unique plan types (since we'll need this for the next question anyway). The resulting table yields `r nrow(plan.type.table)` rows, so there are `r nrow(plan.type.table)` total plan types. 

In [30]:
# Number of unique plan types
n_plan_types = full_ma_data['plan_type'].nunique()
print(f"There are {n_plan_types} unique plan types.")

There are 10 unique plan types.


**3. Provide a table of the count of plans under each plan type.**

In [31]:
# Pivot table to get the number of plans per type and year
plans_per_type = full_ma_data.pivot_table(index='plan_type', columns='year', values='planid', aggfunc='count')
plans_per_type.sort_values(by=2015, ascending=False)

year,2015
plan_type,
Medicare Prescription Drug Plan,991457
Local PPO,704993
HMO/HMOPOS,479275
Employer/Union Only Direct Contract PDP,25630
Regional PPO,17578
PFFS,13658
1876 Cost,7157
MSA,6518
Medicare-Medicaid Plan HMO/HMOPOS,4130


**4. Remove all special needs plans (SNP), employer group plans (eghp), and all "800-series" plans.**

Provide an updated table after making these exclusions.

In [32]:
# Exluce SNP and EGHP plans, and plans between 800 and 899
final_ma_data = full_ma_data[(full_ma_data['snp'] == "No") & (full_ma_data['eghp'] == "No") & ((full_ma_data['planid'] < 800) | (full_ma_data['planid'] >= 900))]
plans_per_type = final_ma_data.pivot_table(index='plan_type', columns='year', values='planid', aggfunc='count')
plans_per_type.sort_values(by=2015, ascending=False)

year,2015
plan_type,
Medicare Prescription Drug Plan,269153
HMO/HMOPOS,36588
Local PPO,16728
Regional PPO,8531
1876 Cost,6329
PFFS,4232
Medicare-Medicaid Plan HMO/HMOPOS,4130
National PACE,1216
MSA,232


**5. Provide a table of the average enrollments for each plan type in 2015.**

In [37]:
# Pivot table to get the average enrollment per plan type and year
enrollment_per_type = final_ma_data.pivot_table(index='plan_type', columns='year', values='avg_enrollment', aggfunc='mean')
enrollment_per_type.round(0).astype('int').sort_values(by=2015, ascending=False)

year,2015
plan_type,
HMO/HMOPOS,849
Medicare-Medicaid Plan HMO/HMOPOS,624
Medicare Prescription Drug Plan,312
Local PPO,311
1876 Cost,228
Regional PPO,202
National PACE,140
PFFS,125
MSA,108
